In [ ]:
import requests
import base64
from os import listdir
from os.path import isfile, join
import numpy as np
import json

# Read 100 images of cancer
# Make predictions and store the prediction and probability

def get_prediction(image_data):
    url = 'https://drzg5t7mqj.execute-api.us-east-1.amazonaws.com/Predict/1560f5df-863a-41fc-b008-66b28992ab82'
    r = requests.post(url, data=image_data)
    response_raw = getattr(r,'_content').decode("utf-8")
    response = json.loads(response_raw)
    #print(response)
    return response

path_cancer = "C:/Users/shriy/OneDrive/Pictures/images/9022/1"

cancer_images_list = [f for f in listdir(path_cancer) if isfile(join(path_cancer, f))]
num_samples = len(cancer_images_list)
prediction_cancer = np.zeros((num_samples,2))
for a in range(0,num_samples):
    with open(path_cancer + '/' + cancer_images_list[a], "rb") as image:
        image_cancer = base64.b64encode(image.read())
    output = get_prediction(image_cancer)
    if (output['predicted_label'] == 'Benign(0)'):
        prediction_cancer[a,0] = 0
        prediction_cancer[a,1] = 1 - output['score']
    else:
        prediction_cancer[a,0] = 1
        prediction_cancer[a,1] = output['score']

# Read 100 images of healthy
# Make predictions and store the prediction and probability
path_healthy = "C:/Users/shriy/OneDrive/Pictures/images/9022/0"

healthy_images_list = [f for f in listdir(path_healthy) if isfile(join(path_healthy, f))]

prediction_healthy = np.zeros((num_samples,2))
for a in range(0,num_samples):
    with open(path_healthy + '/' + healthy_images_list[a], "rb") as image:
        image_healthy = base64.b64encode(image.read())
    output = get_prediction(image_healthy)
    if (output['predicted_label'] == 'Benign(0)'):
        prediction_healthy[a,0] = 0
        prediction_healthy[a,1] = output['score']
    else:
        prediction_healthy[a,0] = 1
        prediction_healthy[a,1] = 1-output['score']

    
     


confusion_matrix = np.zeros((2,2))
confusion_matrix[0,0] = np.sum(prediction_cancer[:,0])
confusion_matrix[1,1] = num_samples - np.sum(prediction_healthy[:,0])
confusion_matrix[0,1] = num_samples - confusion_matrix[0,0]
confusion_matrix[1,0] = np.sum(prediction_healthy[:,0])

print('confusion matrix: ', confusion_matrix)
print('accuracy: ', np.sum(np.trace(confusion_matrix))/np.sum(confusion_matrix))

y_test = np.concatenate((np.ones((num_samples,1)),np.zeros((num_samples,1))),axis=0)
y_score = np.concatenate((prediction_cancer[:,1].reshape(-1,1),1-prediction_healthy[:,1].reshape(-1,1)),axis=0)

# ROC curve
# Compute ROC curve and ROC area for each class
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = 2
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test, y_score)
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area


plt.figure()
lw = 2
class_cancer = 1
plt.plot(fpr[class_cancer], tpr[class_cancer], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[class_cancer])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()
